# Automation

In the next two class sessions we will be talking about automation. A lot falls under that word- including web scraping and GUI interaction (forms & prompts especially) which we have already covered a bit, but for the sake of brevity this is what we will be covering in these two sessions:
* Scheduling
* The use of APIs
* Sending pre-canned emails
* File system management
* System administration tasks

First, we will work on scheduling a program to run a certain operation every day at a certain time, and running the script as a background process so that it can do so. 

We will be building a rudimentary weather tracker in Python that gets the temperature and forecast for the day every day at a certain time. 

### Installing requirements
We have some pre-requisites. We will want to run `pip install schedule requests` to make sure we have access to what we need in Python. 

### Get an API Key
You can register for an OpenWeatherMap account here: https://home.openweathermap.org/users/sign_up
Once you sign up you can sign up for a free tier. The API key will appear under https://home.openweathermap.org/api_keys after you have registered. The first 1,000 calls a day is free. You can make sure that you don't go over 1,000 a day by setting a quota on your account in your subscriptions: https://home.openweathermap.org/subscriptions
By default, the limit is 2,000 calls. $.0012/call * 1,000 calls (above the 1,000 free) is 1.2GBP per day if you don't change the limit and somehow hit the limit. Our script only calls once a day, so you won't. You can always end the subscription, or set the limit to 1,000 to never go over the free limit.

For the sake of this class, I will share my API key with you all, since I have it limited. The key will be destroyed tomorrow. 

### Store the API Key as an environment variable
Note: It is important to <b> NEVER leave your API keys exposed in your code </b> if you publish the code anywhere on the internet, like GitHub. People have built scrapers for GitHub where they search for exposed API keys in particular and abuse your account. If you set your limit to 1,000, you won't be charged even if the key does get published, but it is still poor practice and why let someone use the resource you registered for? 

How do we get around this? We store our API key as an environment variable in our local operating system. This is easy to do and much more secure. Let's call our API key API_KEY for consistency (environment variables are traditionally all capitalized like this, as they are constants). There are instructions on how to set an environment variable in [Windows](https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/set_1), [Linux](https://www.freecodecamp.org/news/how-to-set-an-environment-variable-in-linux/), and [Mac OS X](https://youngstone89.medium.com/setting-up-environment-variables-in-mac-os-28e5941c771c)

If you eventually wanted to run something like this script in GitHub actions (and schedule via a workflow file instead), you can add [secrets](https://docs.github.com/en/actions/security-guides/using-secrets-in-github-actions) within your repository. They act the same way. The code used to do the work is public, but the API keys are not. 

### Imports

In [3]:
import time
import requests
import os
import json
from datetime import datetime
import schedule

### Fetch Weather Method

In [4]:
def fetch_weather():
    api_key = os.environ.get('WEATHER_KEY')
    city = 'Sarajevo'
    country_code = 'BA'

    # API request URL with units=metric for Celsius
    api_url = f'http://api.openweathermap.org/data/2.5/weather?q={city},{country_code}&appid={api_key}&units=metric'

    # Sending request to the API
    response = requests.get(api_url)

    if response.status_code == 200:
        weather_data = response.json()

        # Extracting relevant information (e.g., temperature, description)
        temperature = weather_data['main']['temp']
        description = weather_data['weather'][0]['description']

        # Storing data in a file with timestamp
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        with open('weather_log.txt', 'a') as file:
            file.write(f'{timestamp} - Temperature: {temperature}°C, Description: {description}\n')
    else:
        print(f"Failed to fetch weather data. Status Code: {response.status_code}")

### Schedule the weather fetch

In [5]:
# Schedule the job to run every day at 19:20
schedule.every().day.at("19:20").do(fetch_weather)

Every 1 day at 19:20:00 do fetch_weather() (last run: [never], next run: 2023-12-05 19:20:00)

### Run the Scheduler

You won't want to do this in the Jupyter notebook generally, this is just here to breakdown the code. I'll show you how to run the entire script as a [background process](https://www.howtogeek.com/440848/how-to-run-and-control-background-processes-on-linux/). 
This works a bit different in Windows and I haven't tried this as I don't have a Windows machine, so please let me know [if it works](https://stackoverflow.com/questions/32808730/running-python-script-as-a-windows-background-process). 

In [ ]:
# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)